José Delgado

In [19]:
import dill
import os

In [20]:
fit_models = {}
models = ['rf', 'gb', 'ridge', 'sgd']

In [21]:
model_files = [f"models/{model}_gridsearch.pkl" for model in models]

In [22]:
model_files

['models/rf_gridsearch.pkl',
 'models/gb_gridsearch.pkl',
 'models/ridge_gridsearch.pkl',
 'models/sgd_gridsearch.pkl']

In [23]:
for model_file in model_files:
    if os.path.exists(model_file):
        with open(model_file, 'rb') as f:
            model_name = os.path.basename(model_file).replace('_gridsearch.pkl', '')
            fit_models[model_name] = dill.load(f)

In [24]:
fit_models

{'rf': GridSearchCV(cv=10,
              estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                        ('pca', PCA(n_components=45)),
                                        ('randomforestclassifier',
                                         RandomForestClassifier(random_state=1234))]),
              n_jobs=-1,
              param_grid={'randomforestclassifier__max_depth': [None, 5, 7, 9],
                          'randomforestclassifier__n_estimators': [50, 100, 200,
                                                                   300, 400]}),
 'gb': GridSearchCV(cv=10,
              estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                        ('pca', PCA(n_components=45)),
                                        ('gradientboostingclassifier',
                                         GradientBoostingClassifier(random_state=1234))]),
              n_jobs=-1,
              param_grid={'gradientboosti

# 7. Evaluate

## Calculate Classification Metrics

## Review Confusion Matrix

# 8. Deploy

## Save Model

## Load Model and PCA

## Call New Data to Be Scored

## Make Predictions

## Check if Predictions Table Exists, If Not Create It

## Push back into Database

## View Loaded Results